# 1. 모델 구조와 작동 과정
## 모델 구조
결정 나무는 이름에서 알 수 있듯이 그 구조가 나무 구조(tree-structured)인 분류 및 회귀 모델 입니다.
- 잎 노드를 제외한 모든 노드는 가지(branch)를 통해 두 개의 노드로 분지됨
- 각 가지에는 노드를 분리하는 조건이 있고 각 노드에는 해당 조건을 만족하는 학습 샘플이 포함됨
- 더 이상 분지되지 않는 최하위 노드인 잎 노드에 속한 학습 샘플을 바탕으로 라벨에 대한 예측을 수행함

# 2. 모델 특성
## 수직/수평 공간 분할
결정 나무는 데이터 공간을 수직이나 수평으로만 분할하는 특성이 있습니다.
- 각 가지에 부여된 조건의 구조가 특징 x랑 상수 c를 비교하는 x<c 의 구조이기 때문
- 결정 나무는 데이터 공간을 비선형적으로 분리할 수 없어, 기존 특징을 변환하거나 새로운 특징을 생성해야 함

## 높은 설명력
결정 나무는 모델의 작동 과정을 IF - THEN 규칙 집합 형태로 손쉽게 설명할 수 있으며, 규칙의 근거 샘플 수와 정확도도 알 수 있습니다.
- 이러한 설명력 덕분에 결정 나무는 대출 연체 여부 예측 등 설명력이 필요한 과제에 많이 사용함
- 심지어는 지도 학습 과제가 아니라 특정한 이벤트의 발생 조건을 판단하는 데도 사용함

## 이진화
결정 나무는 모든 종류의 특징을 이진화 합니다.
- 연속형 특징 x에 대해서도 c보다 작거나 같은 지만 판단하므로, x를 사용한 것과 x<c 인지를 나타내는 이진형 특징을 사용한 것과 같은 효과를 냄
- 연속형 특징이 가진 정보가 거의 무시된다는 문제가 있음
- 특징이 모두 이진화 되기 때문에 다른 모델에 비해 전처리가 가장 적게 필요함
  - 스케일링 전에는 x<c라는 조건이 스케일링 후에는 x' < c' 로 스케일만 바뀌므로 스케일링할 필요가 없음
  - 더미화를 하지 않더라도 x == c라는 형태의 조건을 통해 더미화를 한 것처럼 모델이 작동함
  - 결측을 따로 대체하지 않더라도 결측 자체를 값으로 간주하여 x == nan 이라는 형태의 질문을 만들 수도 있습니다.

# 3. 주요 하이퍼 파라미터
## 깊이
깊이가 깊어질수록 가지가 더 많아 데이터 공간이 더 잘게 쪼개지므로, 깊이와 복잡도는 정비례한다고 할 수 있습니다.
- 결정 나무의 모델 복잡도는 깊이로 결정된다고 하더라도 과언이 아님
- 사이킷런에는 max_depth, min_samples_leaf, min_impurity_decrease 등 복잡도와 관련된 다양한 하이퍼 파라미터가 있으나 모두 깊이를 결정하기 위한 것임

## 클래스 가중치
클래스 가중치를 설정함으로써 비용 민감 모델로 변환할 수 있음

$\hat{y}=
\begin{cases}
1,\;if\;w_1n_1>w_0n_0\\
0,\;\;\,otherwise
\end{cases}$

- $w_1$ : $n_1$에 대한 가중치 (보통 클래스 불균형 비율로 설정)
- $w_0$ : $n_0$에 대한 가중치 (보통 1로 설정)

## 분지 기준
분기 기준에 따른 모델의 성능은 큰 차이가 없다고 알려져 있습니다.
- 결정 나무의 학습은 각 노드의 불순도(impurity)를 최소화하는 조건을 찾아 노드를 분지하는 것이라 할 수 있음
- 불순도란 각 노드에 속한 라벨의 편차라고 할 수 있음
  - (분류) 라벨이 한 클래스의 값을 주로 가질 때 불순도가 낮음
  - (회귀) 라벨의 표준편차가 작을 때 불순도가 낮음
- 불순도를 측정하는 척도로 엔트로피와 지니 지수 등이 있으니, 측정 방법에 따른 모델의 성능은 큰 차이가 없다고 알려져 있음
  - 대표적인 하이퍼 파라미터지만 측정 방법에 따른 성능 차이가 크지 않아 하이퍼 파라미터 튜닝 대상에 포함할 필요가 없음

# 4. 사이킷런 실습
## 예제 데이터 불러오기
결정 나무를 학습할 예제 데이터를 불어 옵니다.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('../data/classification/glass6.csv')
X = df.drop('y', axis=1)
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2022)

## 모델 학습
결정 나무는 tree 모듈의 DecisionTreeClassifier와 DecisionTreeRegressor 클래스를 이용해 구현할 수 있습니다.

주요 인자
- max_depth
  - 최대 깊이
  - 기본값 : None
    - 기본값으로 설정되면 잎 노드에 min_samples_split보다 작은 샘플 수가 포함될 때까지 분지하여 최대 깊이까지 학습합니다.
- min_samples_split
  - 노드를 분지할 수 있는 노드 내 최소 샘플 수
  - 기본값: 2
- class_weight
  - 클래스 가중치로 키가 클래스 값이 가중치인 사전
  - 기본값 : None
    - 기본값으로 설정되면 각 클래스에 대한 가중치를 모두 1로 설정합니다.

결정 나무 모델 학습 예제

In [2]:
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.metrics import *
model = DTC(random_state=2022).fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
print(acc, rec)

0.8888888888888888 0.5454545454545454


- 라인3 : 결정 나무를 분지할 때 모든 조건을 탐색하지 않고 임의로 일부만 탐색하므로 실행시마다 결과가 달라질 수 있습니다. 이를 방지하기 위해서 씨드를 고정했습니다.

## 전처리의 불필요 확인
앞서 설명한대로 스케일링할 필요가 없는지를 확인해보겠습니다.

전처리의 불필요성 확인 예제

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X_train)
Z_train = scaler.transform(X_train)
Z_test = scaler.transform(X_test)
model2 = DTC(random_state=2022).fit(Z_train, y_train)
y_pred = model2.predict(Z_test)
acc = accuracy_score(y_test,y_pred)
rec = recall_score(y_test,y_pred)
print(acc,rec)

0.8888888888888888 0.5454545454545454


## 클래스 가중치 설정
클래스 가중치를 설정해서 비용 민감 모델을 학습하겠습니다

클래스 불균형 비율 계산 예제

In [5]:
num_majority_sample = y_train.value_counts().iloc[0]
num_minority_sample = y_train.value_counts().iloc[-1]
class_imbalance_ratio = num_majority_sample / num_minority_sample
print(class_imbalance_ratio)

8.352941176470589


- 라인 1
  - value_counts 메서드를 이용해 다수 클래스 샘플 수를 계산합니다. 
  - value_counts 메서드는 시리즈 요소의 출현 빈도를 내림차순으로 정렬하므로 0번째에 나오는 값이 다수 클래스의 샘풀 수가 됩니다.
  

- 라인 2
  - 라인 1과 비슷한 방법으로 소수 클래스 샘플 수를 계산 했습니다.

- 라인 3
  - 다수 클래스 샘플 수를 소수 클래스 샘플 수로 나눠 클래스 불균형 비율을 계산

클래스 가중치 적용 예제

In [6]:
model3 = DTC(random_state=2022, class_weight={0:1,1:class_imbalance_ratio}).fit(X_train, y_train)
y_pred = model3.predict(X_test)
acc = accuracy_score(y_test,y_pred)
rec = recall_score(y_test,y_pred)
print(acc, rec)

0.8888888888888888 0.5454545454545454


- 라인 1
  - 클래스 0의 가중치를 1로, 클래스 1의 가중치를 class_imbalance_ratio로 설정


- 성능에 전혀 변화가 없음
- 가중치를 더 크게 부여해야 함

클래스 가중치를 설정해서 비용 민감 모델을 학습하겠습니다.

클래스 가중치 적용 예제

In [7]:
model4 = DTC(class_weight={0:1,1:class_imbalance_ratio*100}).fit(X_train, y_train)
y_pred = model4.predict(X_test)
acc = accuracy_score(y_test,y_pred)
rec = recall_score(y_test,y_pred)
print(acc, rec)

0.9259259259259259 0.6363636363636364


- 재현율과 정확도가 모두 올랐음
- 이는 소수 클래스의 결정 공간이 확장할 때 주로 발생하는 다수 클래스의 오 뷴류가 발생하지 않았기 때문
- 그러나 일반적으로는 소수 클래스에 대한 가중치를 크게 설정할수록 정확도는 떨어지고 재현율은 올라감